In [0]:
-- =========================================================================================
-- Account Prism View
-- =========================================================================================
-- Joins Tables 01 - 14 from teamconnect.pca_prism Schema
-- * Main Table used
-- ** Support Table used for linking
-- =========================================================================================
--teamconnect.pca_prism.`01_tbl_account_prism_payments`* 
--teamconnect.pca_prism.`02_tbl_account_prism_services`*
--teamconnect.pca_prism.`03_tbl_account_prism_adjustments`*
--teamconnect.pca_prism.`04_tbl_base_btns`*
--teamconnect.pca_prism.`05_tbl_account_prism_sub_trt_prm`*
--teamconnect.pca_prism.`06_tbl_trt_raw_clean`**
--teamconnect.pca_prism.`07_tbl_trt_latest`**
--teamconnect.pca_prism.`08_tbl_prm_shaped`**
--teamconnect.pca_prism.`09_tbl_prm_active`**
--teamconnect.pca_prism.`10_tbl_subinf_cur`**
--teamconnect.pca_prism.`11_tbl_subinf_groups`**
--teamconnect.pca_prism.`12_tbl_subinf_other_accounts`*
--teamconnect.pca_prism.`13_tbl_trt_ltype`*
--teamconnect.pca_prism.`14_tbl_trt_events_12m`*
-- =========================================================================================
--DROP VIEW teamconnect.pca_prism.vw_account_prism;
--CREATE OR REPLACE MATERIALIZED VIEW teamconnect.pca_prism.vw_account_prism AS
REFRESH MATERIALIZED VIEW teamconnect.pca_prism.vw_account_prism;
WITH base_btns AS (
  SELECT btn, dpi_environment
  FROM   teamconnect.pca_prism.04_tbl_base_btns 
)
SELECT
  b.btn,
  b.dpi_environment,

  -- Subscriber / treatment / promos
  s.customer_name,
  s.service_address,
  s.connect_date,
  s.disconnect_date,
  s.days_in_service,
  s.suspend_open_flag,
  s.suspend_date,
  s.dpa_open_flag,
  s.dpa_date,
  s.nsf_open_flag,
  s.nsf_date,
  s.ovr_open_flag,
  s.ovr_date,
  s.last_reminder_date,
  s.last_reminder_amount_due,
  s.last_suspended_amount,
  s.disqualified_upsell_sne_yn,
  s.active_promos,
  s.recent_promos_12m,
  trt_ltype.last_treatment_date,
  trt_ltype.last_treatment_type,
  trt_ltype.last_treatment_type_description,
  trt_ltype.last_treatment_amount,
 

  -- Payments
  COALESCE(p.payments_count_all_time, 0) AS payments_count_all_time,
  COALESCE(p.payments_sum_all_time,  0) AS payments_sum_all_time,
  COALESCE(p.avg_monthly_payment_12m,0) AS avg_monthly_payment_12m,
  COALESCE(p.payments_count_12m,    0)  AS payments_count_12m,
  p.last_payment_date,
  p.last_payment_type,
  p.last_payment_amount,
  p.payment_types_last_12m,
  p.last_12_mo_payments,
  p.payment_ledger_last_12m,

  -- Services
  COALESCE(svc.current_mrc_sum, 0)                        AS current_mrc_sum,
  COALESCE(svc.current_services, ARRAY())                 AS current_services,
  COALESCE(svc.current_month_mrc_sum, 0)                  AS current_month_mrc_sum,
  COALESCE(svc.current_month_itemized_services, ARRAY())  AS current_month_itemized_services,

  -- Adjustments / LOB
  COALESCE(a.adjustments_count_all_time, 0)              AS adjustments_count_all_time,
  COALESCE(a.adjustments_sum_all_time,   0)              AS adjustments_sum_all_time,
  COALESCE(a.adjustments_last_12m,       ARRAY())        AS adjustments_last_12m,
  COALESCE(a.adjustments_current_month_sum, 0)           AS adjustments_current_month_sum,
  COALESCE(a.adjustments_current_month_items, ARRAY())   AS adjustments_current_month_items,
  a.lobtype_last_seen,
  a.lobtype_mode_12m,

  -- Linked accounts (from your Option 2)
  COALESCE(o.other_btns_same_name_addr, ARRAY())         AS other_btns_same_name_addr,

    -- 12-month history array
COALESCE(trt12.treatment_events_last_12m, ARRAY()) AS treatment_events_last_12m,
size(COALESCE(trt12.treatment_events_last_12m, ARRAY())) AS treatment_event_cnt_12m


FROM base_btns b
LEFT JOIN teamconnect.pca_prism.05_tbl_account_prism_sub_trt_prm    s
       ON s.btn = b.btn AND s.dpi_environment = b.dpi_environment
LEFT JOIN teamconnect.pca_prism.01_tbl_account_prism_payments       p
       ON p.btn = b.btn AND p.dpi_environment = b.dpi_environment
LEFT JOIN teamconnect.pca_prism.02_tbl_account_prism_services       svc
       ON svc.btn = b.btn AND svc.dpi_environment = b.dpi_environment
LEFT JOIN teamconnect.pca_prism.03_tbl_account_prism_adjustments    a
       ON a.btn = b.btn
LEFT JOIN teamconnect.pca_prism.12_tbl_subinf_other_accounts        o
       ON o.btn = b.btn AND o.dpi_environment = b.dpi_environment
LEFT JOIN teamconnect.pca_prism.13_tbl_trt_ltype  trt_ltype
       ON trt_ltype.btn = b.btn AND trt_ltype.dpi_environment = b.dpi_environment
LEFT JOIN teamconnect.pca_prism.14_tbl_trt_events_12m trt12
       ON trt12.btn = b.btn AND trt12.dpi_environment = b.dpi_environment;



